# Model Preparation and Processing

Now that our data is usable for our model, we can start
going through the steps to make the first set of
models.

In this notebook, we'll make preliminary and final
models for our predictions. We'll convert our data from
the original data set's final structure into a form we
can train our models on. Rather than having a series of
raw text that we would generally be able to read, we
want our model to receive vectorized data. This applies
to several other features as well, which we will
address as they come up.

## Imports

Many of the imports from the first notebook are used
again alongside several additional packages.

Because the previous notebook is dedicated to the major
preprocessing steps and because that same notebook was
used to create a simplified form of our original data
structures, we'll be using that data as our starting
point. Since that file is much smaller, it has been
provided in the data/ folder in this project's
repository. It can also be recreated from the processes
outlined in
[the previous notebook](./1_Data_Prep.ipynb).

As before, required packages are defined in the yaml
file in the _code/ folder in this repository.

In [1]:
import numpy as np
import polars as pl
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, \
            GradientBoostingRegressor
from sklearn.model_selection import train_test_split, \
            cross_val_score, GridSearchCV
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

from joblib import parallel_backend, Parallel, \
            dump, load

from _code.cleaner import preprocess

from IPython.display import Image, Markdown
sns.set()

As mentioned, we'll import our previously-created data
set rather than any of the raw data sets we used
initially.

In [2]:
cards = pd.read_parquet('./data/simplified_cards.parquet')

## Data Engineering

While we have pretty much all of the data we need from
the previous notebook, we can still benefit from having
several other pieces of information that can be
extracted from the data. One thing that we want to
modify is the way our abilities are stored.

### Processing Abilities/Oracle Text

Our abilities are currently stored as one string per
card. While this is useful in reading our abilities,
there is an important thing to note - a card can have
multiple abilities, as denoted by new lines on cards,
or a `\n` in the oracle text.

Since these abilities are mostly independent of one
another, we don't want them to be pushed together when
we convert them into vectorized versions of themselves.
A card with "Flying" and "When CARDNAME enters the
battlefield" shouldn't be read as "Flying when CARDNAME
enters the battlefield," they aren't part of the same
ability.

To get around this, we'll split each card's Oracle text
into an array at each `\n` and we can vectorize the
text from there.

Let's go ahead and look at the before and after of some
of our cards.

First, we'll generate our `processed_abilities` by
sending the Oracle text through our custom `preprocess`
function, which takes every ability we send it and
breaks it out into individual words, reducing them to
their root words, removing stop words, and then putting
them all back together for us.

After that, we'll separate of the returned abilities
into a list of abilities as described by splitting on
`\n`.

Once we finish that, we'll create a new column that is
a count of the total number of a given card has, as
cards that have more abilities are likely to have more
complexities or power and will, by extension, hold more
value. In theory at least.

In [3]:
display(
    cards[[
        'name','oracle_text',
        'median_normal','median_foil'
        ]].head()
)

processed_abilities = preprocess(cards['oracle_text'])
cards['abilities_list'] = [
    abilities.split('\n') 
    for abilities in processed_abilities
    ]

cards['n_abilities'] = cards['abilities_list'].map(len)

display(
    cards[[
        'name','abilities_list',
        'median_normal','median_foil',
        'n_abilities'
        ]].head()
)


,name,oracle_text,median_normal,median_foil
0,Fury Sliver,All Sliver creatures have double strike.,0.380,3.95
1,Kor Outfitter,"When CARDNAME enters the battlefield, you may ...",0.240,7.78
2,Siren Lookout,"Flying\nWhen CARDNAME enters the battlefield, ...",0.060,0.23
3,Web,Enchant creature (Target a creature as you cas...,0.640,NaN
4,Venerable Knight,"When CARDNAME dies, put a +1/+1 counter on tar...",0.095,0.28


,name,abilities_list,median_normal,median_foil,n_abilities
0,Fury Sliver,[sliver creature double strike],0.380,3.95,1
1,Kor Outfitter,[cardname enters battlefield may attach target...,0.240,7.78,1
2,Siren Lookout,"[fly , cardname enters battlefield explores ]",0.060,0.23,2
3,Web,"[enchant creature , enchant creature get +0/...",0.640,NaN,2
4,Venerable Knight,[cardname die put +1/+1 counter target knight ...,0.095,0.28,1


## Model Preparation

Now that we have our data ready for our modeling steps,
we can go ahead and do our splits for our training and
testing subsets. As mentioned before, we'll be making
sure that our training and testing data have cards from
different sets at similar rates.

We'll also go ahead and get rid of the original
`prices_` columns, as we're only going to focus on the
median prices for this project.

In [4]:
X = cards.drop(columns=['prices_normal','prices_foil'])
y = cards['set']
train, test, _, _ = \
    train_test_split(
        X,y,stratify=y,
        random_state=13,
        test_size=0.2
    )

# we'll reset the indices of both sets to more easily
# translate between polars and pandas in an upcoming
# step

train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

# printing out value counts to demonstrate that the
# splits are even 

display(
    train['set'].value_counts(normalize=True)[:5],
    test['set'].value_counts(normalize=True)[:5]
)

set
mb1      0.027116
plist    0.016935
clb      0.014430
sld      0.013749
j22      0.013388
Name: proportion, dtype: float64

set
mb1      0.027096
plist    0.016995
clb      0.014430
sld      0.013789
j22      0.013388
Name: proportion, dtype: float64

### Ability Vectorization

In order to train a model on our data, we need to have
a feature/column for each word or phrase we want to
target. We'll use the Count Vectorizer provided by
SKlearn to do this.

In [5]:
# Our token pattern needs to be able to account for
# several non-standard things for it to be effective
# for our needs. As normal, it needs to be able to
# match words that are contiguous letters of an
# arbitrary length. However, we also need to be able to
# account for numbers in a few formats. These can be
# wrapped in curly brackets, e.g. {2}, representing 2
# colorless mana.
# It also needs to be able to account for letters
# inside of curly brackets and return them as such.
# It must also match something like +1/+1, -1/-1, or
# several other variations thereof.
# Lastly, it should also ignore any text that is inside
# of parentheses, as this text is reminder text - which
# is text that explains what an ability does but this
# text doesn't actually contribute to explaining what
# a card does in a meaningful way. 

token_pattern = \
    r"([a-zA-Z]+(?:’[a-z]+)?|[+-]?\d\/[+-]?\d|\{\d\d?\}|\{.\s?.?\}|\n)|\(.+?\)"

cvec = CountVectorizer(
        token_pattern=token_pattern,
        # min_df=0.0005, # <= this will mean that the
                    # minimum number of cards that it
                    # takes for an ability to show up
                    # on this list will be 46 after the
                    # explode is run, since it will be
                    # 83,000 entries long. We'll just
                    # limit our overall features since
                    # this is such a small percentage. 
        max_df=0.4,
        ngram_range=(1,5),
        max_features=1500
    )

Currently, our abilities are listed out as lists or
arrays, but the vectorizer needs them as strings rather
than a list of strings. To effectively accomplish this,
we'll need to explode our data frame. This will make
each card have a duplicate entry for each ability it
has, so cards that have 3 or 4 abilities will instead
have 3 or 4 entries with 1 ability each. Because of
this, our data frame will be substantially larger. We
won't be able to reliably use a minimum document
frequency in our vectorizer, but that's okay.

In [6]:
display(train[['name','abilities_list']].head())
explode_train = train.explode('abilities_list')
display(explode_train[['name','abilities_list']].head())

,name,abilities_list
0,Stolen Identity,"[create token copy target artifact creature , ..."
1,Ageless Entity,[whenever gain life put many +1/+1 counter car...
2,Izzet Locket,"[{t} add {u} {r} , u r u r u r u r {t} sacrif..."
3,Inspired Charge,[creature control get +2/+1 end turn]
4,"Oros, the Avenger","[fly , whenever cardname deal combat damage p..."


,name,abilities_list
0,Stolen Identity,create token copy target artifact creature
0,Stolen Identity,cipher
1,Ageless Entity,whenever gain life put many +1/+1 counter card...
2,Izzet Locket,{t} add {u} {r}
2,Izzet Locket,u r u r u r u r {t} sacrifice cardname draw t...


Now we can perform our vectorization.

In [7]:
explode_vec = cvec.fit_transform(
        explode_train['abilities_list']
    )
explode_vec = pd.DataFrame.sparse.from_spmatrix(
    explode_vec
)

# we save the vocab here to export for later. This is
# so we can bring in new data and make sure it's only
# being segmented into vocab we can "understand"
explode_vec.columns = sorted((vocab := cvec.vocabulary_))
explode_vec['id'] = explode_train['id'].values
explode_vec.head()

,+1/+0,+1/+0 end,+1/+0 end turn,+1/+1,+1/+1 counter,+1/+1 counter cardname,+1/+1 counter creature,+1/+1 counter creature control,+1/+1 counter target,+1/+1 counter target creature,...,{u} {t},{u} {u},{w},{w} cardname,{w} {b},{w} {t},{w} {u},{w} {w},{x},id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2831fe77-ea98-4334-a78e-01580fb002c0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2831fe77-ea98-4334-a78e-01580fb002c0
2,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f2d9b77b-d775-4546-91c8-df438a1d8dbe
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a32ecc71-b924-4414-94ee-c6cb0ba752e5
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a32ecc71-b924-4414-94ee-c6cb0ba752e5


Let's take a look at the number of total entries versus
the number of unique cards in our data set.

In [8]:
explode_vec['id'].nunique(),explode_vec.shape[0]

(49896, 88164)

It looks like we have a little bit under 2 abilities
per card, which makes sense considering most cards have
1 ability.

For this next step, we'll convert the data frame we
have from a Pandas data frame into a Polars data frame.
The rest of the processes afterwards will take place in
Pandas, but Polars is able to much more efficiently
handle group-by operations, and we need to have a sum
of each individual word for each unique card.

In [9]:
# convert pandas vectorized dataframe to polars
pl_vec = pl.from_pandas(explode_vec.astype(np.int32,errors='ignore'))
# perform group by and sum aggregation and convert back
# to pandas 
agged_vec = pl_vec.groupby('id').sum().to_pandas()

agged_vec.shape

(49896, 1501)

Great. Now we have our data frame back to on entry per
card. We'll keep this data frame to the side for now so
that we can merge it back in later. There are a couple
more vectorizations that need to take place.

### Type Line Vectorization

A much simpler type of categorization for cards is the
type that a card is. Creatures and enchantments are
cast differently, have different drawbacks and
benefits, and sometimes are both on the same card. We
can capture a lot of information just in observing a
card's types and super types. For now, we'll ignore
subtypes because there is a lot more nuance in those
and they generally don't change much of how a card
works. There are a few exceptions but even then they're
minor.

We'll do much of the same process by vectorizing our
types through a count vectorizer, effectively one hot
encoding our data but in a little bit simpler of a
process. We don't need any special patterns or limits
here because the card types are all single words.

The only pre-processing we need to do on this subset of
information is to separate a card's types from its
subtypes. On Magic cards, a card's type line is
divided into its types (and supertypes) and subtypes by
an emdash (—). Since this is a lambda apply process,
we actually want to use polars to handle this process
more efficiently. Then we can give it an ID column and
set it aside just like we did with the Oracle text
before.

In [10]:
card_type_cvec = CountVectorizer()

type_frame = pl.from_pandas(train['type_line']).apply(lambda x: x.split('—')[0])
type_df = pd.DataFrame.sparse.from_spmatrix(
    card_type_cvec.fit_transform(type_frame)
)
# we save the type vocab for later to export
type_df.columns = sorted((type_vocab:=card_type_cvec.vocabulary_))
type_df['id'] = train['id']
display(type_df.head(),type_df.shape)

,artifact,basic,conspiracy,creature,enchantment,instant,land,legendary,planeswalker,snow,sorcery,stickers,tribal,world,id
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2831fe77-ea98-4334-a78e-01580fb002c0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,f2d9b77b-d775-4546-91c8-df438a1d8dbe
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,a32ecc71-b924-4414-94ee-c6cb0ba752e5
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,98151986-90de-4f76-9abd-507039e7c9ad
4,0,0,0,1,0,0,0,1,0,0,0,0,0,0,c2790685-f6ae-4106-ae4d-fe97954bcb82


(49896, 15)

This actually shows us that there are a couple of cards
that have types that we aren't interested in capturing
still. Sticker cards and Conspiracy cards make up very
specific portions of sets. They only exist in one and
two sets respectively. They have very unique abilities
that generally don't fall very well in line with the
rest of Magic as a whole. We'll want to keep this in
mind later when we bring all of this back together.

### Color Identity Vectorization

A feature that holds a lot of importance for a group of
people that have a lot of influence on the market is
color identity. The Commander format is one of the most
played formats in Magic and a card's ability to slot
into specific commander decks can be a huge indicator
for whether or not it's usable in the format.

The color identity is already split into a list for us
in our data set, so we can just merge those lists into
strings and run another count vectorizer that only
looks for single letters to vectorize on.

One thing to keep in mind - is that a card without a
color identity is considered colorless. This has its
own unique properties in Magic, so instead of just
treating it as a card that has none of the other
properties, we want to treat it as a card that
specifically is colorless.

Just like before, we'll give our data frame an ID
column to merge with later.

In [11]:
display(train[['color_identity']].head())

train['str_color_identity'] = \
    train['color_identity'].map(
        lambda x: ' '.join(x)
        )

display(train[['str_color_identity']].head())

color_match = CountVectorizer(
    token_pattern=r"[wubrg]"
)
color_id_df = pd.DataFrame.sparse.from_spmatrix(
    color_match.fit_transform(train['str_color_identity'])
)

# we don't have to save this vocabulary because there
# are only 5 possible options for color in Magic 
color_id_df.columns = sorted(color_match.vocabulary_)
color_id_df['c'] = color_id_df.T.apply(lambda x: 1 if sum(x)==0 else 0)
color_id_df['id'] = train['id']

display(color_id_df.head())

,color_identity
0,[U]
1,[G]
2,"[R, U]"
3,[W]
4,"[B, R, W]"


,str_color_identity
0,U
1,G
2,R U
3,W
4,B R W


,b,g,r,u,w,c,id
0,0,0,0,1,0,0,2831fe77-ea98-4334-a78e-01580fb002c0
1,0,1,0,0,0,0,f2d9b77b-d775-4546-91c8-df438a1d8dbe
2,0,0,1,1,0,0,a32ecc71-b924-4414-94ee-c6cb0ba752e5
3,0,0,0,0,1,0,98151986-90de-4f76-9abd-507039e7c9ad
4,1,0,1,0,1,0,c2790685-f6ae-4106-ae4d-fe97954bcb82


### Pseudo-Number Vectorization

Something interesting about some of the features we
have is that, at first glance, they appear to be
numbers. Features like power, toughness, and loyalty
all are (mostly) numbers, but there's a very major
distinction between having a 0 in one of these
categories and having a None in them. Because of this,
we have to actually split the columns out into dummies.

While we're doing this step, we'll go ahead and break
out rarity into its own columns since those also have
to be encoded out.

In [12]:
train[['rarity','power','toughness','loyalty']].head()

,rarity,power,toughness,loyalty
0,rare,None,None,None
1,rare,4,4,None
2,common,None,None,None
3,common,None,None,None
4,rare,6,6,None


In [13]:
dummies = pd.get_dummies(
    train[['rarity','power','toughness','loyalty']]
    )
dummies['id'] = train['id']
dummies.head()

,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,power_*,power_-1,power_0,power_1,power_1+*,...,toughness_9,loyalty_0,loyalty_2,loyalty_3,loyalty_4,loyalty_5,loyalty_6,loyalty_7,loyalty_X,id
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2831fe77-ea98-4334-a78e-01580fb002c0
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,f2d9b77b-d775-4546-91c8-df438a1d8dbe
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,a32ecc71-b924-4414-94ee-c6cb0ba752e5
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,98151986-90de-4f76-9abd-507039e7c9ad
4,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,c2790685-f6ae-4106-ae4d-fe97954bcb82


Before we go on to merging everything, let's make sure
that all of our data frames are the same size and that
we didn't miss anything as far as accidental creation.

In [14]:
print(
f"""Colors:     {
    color_id_df.shape[0]
}
Types:      {
    type_df.shape[0]
}
Abilities:  {
    agged_vec.shape[0]
}
Everything: {
    train.shape[0]
}
Dummies:    {
    dummies.shape[0]
}"""
)

Colors:     49896
Types:      49896
Abilities:  49896
Everything: 49896
Dummies:    49896


### Date Conversion

Several of our modeling methods aren't going to like
having our date as a string, so we'll convert this over
into the age of a card in days so we can get rid of the
`released_at` column in our next step.

In [15]:
_now = pd.Timestamp.today().floor('D')
train['card_age'] = train['released_at'].apply(lambda x: (_now - x).days)
display(_now,
    train[['released_at','card_age']].head())

Timestamp('2023-05-26 00:00:00')

,released_at,card_age
0,2022-04-29,392
1,2004-02-06,7049
2,2019-08-23,1372
3,2010-07-16,4697
4,2007-02-02,5957


## Additional Data Reduction

Now that we've converted a lot of data into other
types or forms, we'll take a look at what's in our base
data frame and then reduce it down to just the data we
want to look at currently. At this point, there are
still some features in our data set that we may want to
investigate at some point in the future, but now we
need to gear it even more specifically to the model we
intend to produce.

First, let's look at what is still here before we make
our last feature selection.

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49896 entries, 0 to 49895
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  49896 non-null  object        
 1   oracle_id           49896 non-null  object        
 2   name                49896 non-null  object        
 3   released_at         49896 non-null  datetime64[ns]
 4   image_uris          49896 non-null  object        
 5   mana_cost           49896 non-null  object        
 6   cmc                 49896 non-null  float64       
 7   type_line           49896 non-null  object        
 8   oracle_text         49896 non-null  object        
 9   power               24195 non-null  object        
 10  toughness           24195 non-null  object        
 11  colors              49896 non-null  object        
 12  color_identity      49896 non-null  object        
 13  keywords            49896 non-null  object    

A lot of these features may have a lot of things that
could be interesting, but we need to bring it down to
size for a more limited scope to get some initial
models out.

In [17]:
used_columns = [
    'id',
    # 'oracle_id',
    # 'tcgplayer_id',
    # 'name',
    # 'released_at',  # This has been converted to card age
    # 'image_uris',
    # 'mana_cost',    # may need additional processing.
                    # This isn't something we can
                    # effectively use at this point so
                    # we'll get rid of it now.                 
    'cmc',
    # 'type_line',    # this is already captured as types
    # 'oracle_text',  # this has already been extracted
                    # as abilities
    # 'power',        # vectorized above
    # 'toughness',    # vectorized above
    # 'colors',
    # 'color_identity', # already vectorized
    # 'keywords',     # <= part of the abilities we
                    # extracted, may be useful to look
                    # at at another point 
    # 'legalities',   # <= since we're only looking at
                    # prices for new cards in this
                    # exploration, we can ignore the
                    # legalities of cards for now. This
                    # may be a good thing to look at in
                    # next steps. 
    # 'reserved',
    # 'foil',         # Both foil and nonfoil are
    # 'nonfoil',      # captured in price data
    'promo',
    'reprint',
    # 'variation',
    # 'set_id',       # This section may be useful for
    # 'set',          # later iterations. Ignoring for
    # 'set_type',     # now.
    # 'rarity',       # vectorized above
    'full_art',
    'textless',
    # 'edhrec_rank',  # This may actually be a reall
                        # useful metric to look at, but it
                        # will have to come into play at a
                        # later time. 
    # 'related_uris',
    # 'promo_types',
    # 'loyalty',          # vectorized above
    # 'produced_mana',  # Unsure of the effectiveness
                        # of this, but it's worth
                        # looking into later
    # 'variation_of',
    # 'abilities_list',   # This has already been
                        # captured by our explode.
    'n_abilities',
    'median_normal',    # Need to keep targets
    'median_foil',
    'card_age'
]

In [18]:
train_reduced = train[used_columns].copy()
train_reduced.shape

(49896, 10)

## Bringing the Data Together

Now that we have all of the data we need to be
vectorized vectorized, we can put all of our data
together into a single large data frame.